In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pandas geopandas nasa_power tqdm


In [ ]:
!pip install geopandas pandas numpy tqdm requests tenacity


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_fixed


In [ ]:
!pip install geopandas shapely fiona
import geopandas as gpd
import pandas as pd
import requests
import time
from functools import wraps


In [ ]:
import geopandas as gpd

counties = gpd.read_file("/content/drive/MyDrive/shapefiles/tl_2018_us_county.shp")

counties = counties[["STATEFP", "COUNTYFP", "NAME", "geometry"]]

# Compute centroids
counties["centroid"] = counties.geometry.centroid
counties["lat"] = counties.centroid.y
counties["lon"] = counties.centroid.x

counties.head()


/tmp/ipython-input-851459857.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["centroid"] = counties.geometry.centroid
/tmp/ipython-input-851459857.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["lat"] = counties.centroid.y
/tmp/ipython-input-851459857.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["lon"] = counties.centroid.x


,STATEFP,COUNTYFP,NAME,geometry,centroid,lat,lon
0,31,039,Cuming,"POLYGON ((-97.01952 42.0041, -97.01952 42.0049...",POINT (-96.7874 41.9164),41.916403,-96.787400
1,53,069,Wahkiakum,"POLYGON ((-123.43639 46.2382, -123.44759 46.24...",POINT (-123.43347 46.29113),46.291134,-123.433470
2,35,011,De Baca,"POLYGON ((-104.56739 33.99757, -104.56772 33.9...",POINT (-104.41196 34.34241),34.342414,-104.411958
3,31,109,Lancaster,"POLYGON ((-96.91075 40.78494, -96.91075 40.790...",POINT (-96.68776 40.78417),40.784174,-96.687756
4,31,129,Nuckolls,"POLYGON ((-98.27367 40.0894, -98.27367 40.0894...",POINT (-98.04718 40.17638),40.176380,-98.047185


In [ ]:
def retry(tries=5, delay=1):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(tries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Retry {i+1}/{tries} failed: {e}")
                    time.sleep(delay)
            raise Exception("All retries failed.")
        return wrapper
    return decorator


In [ ]:
@retry(tries=5, delay=1)
def get_nasa_weather(lat, lon, start, end):
    url = "https://power.larc.nasa.gov/api/temporal/daily/point"

    variables = ",".join([
        "T2M",
        "T2M_MAX",
        "T2M_MIN",
        "RH2M",
        "ALLSKY_SFC_SW_DWN"
    ])

    params = {
        "parameters": variables,
        "community": "AG",
        "longitude": lon,
        "latitude": lat,
        "start": start,
        "end": end,
        "format": "JSON"
    }

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    records = data["properties"]["parameter"]
    df = pd.DataFrame(records)
    df.index.name = "date"
    df.reset_index(inplace=True)
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

    return df


In [ ]:
counties = gpd.read_file("/content/drive/MyDrive/shapefiles/tl_2018_us_county.shp")

# Get centroid lat/lon
counties["lat"] = counties.geometry.centroid.y
counties["lon"] = counties.geometry.centroid.x

counties = counties[["STATEFP", "COUNTYFP", "NAME", "lat", "lon"]]
counties.head()


/tmp/ipython-input-3175529622.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["lat"] = counties.geometry.centroid.y
/tmp/ipython-input-3175529622.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["lon"] = counties.geometry.centroid.x


,STATEFP,COUNTYFP,NAME,lat,lon
0,31,039,Cuming,41.916403,-96.787400
1,53,069,Wahkiakum,46.291134,-123.433470
2,35,011,De Baca,34.342414,-104.411958
3,31,109,Lancaster,40.784174,-96.687756
4,31,129,Nuckolls,40.176380,-98.047185


In [ ]:
import requests
import pandas as pd
import time
from typing import Dict, List, Optional
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class NASAPowerDataFetcher:
    """Fetch monthly climate data from NASA POWER API for US counties."""

    BASE_URL = "https://power.larc.nasa.gov/api/temporal/monthly/point"

    def __init__(self, max_retries: int = 3, retry_delay: int = 5):
        """
        Initialize the fetcher.

        Args:
            max_retries: Maximum number of retry attempts for failed requests
            retry_delay: Delay in seconds between retries
        """
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.session = requests.Session()

    def get_us_county_centroids(self, shapefile_path: str) -> pd.DataFrame:
        """
        Get US county centroids from shapefile.

        Args:
            shapefile_path: Path to the county shapefile

        Returns:
            DataFrame with columns: county, state_fp, county_fp, latitude, longitude
        """
        import geopandas as gpd

        logger.info(f"Loading county centroids from {shapefile_path}")

        # Read shapefile
        counties = gpd.read_file(shapefile_path)

        # Get centroid lat/lon
        counties["latitude"] = counties.geometry.centroid.y
        counties["longitude"] = counties.geometry.centroid.x

        # Select and rename columns
        counties_df = counties[["STATEFP", "COUNTYFP", "NAME", "latitude", "longitude"]].copy()
        counties_df.columns = ["state_fp", "county_fp", "county", "latitude", "longitude"]

        logger.info(f"Loaded {len(counties_df)} counties")

        return counties_df

    def fetch_climate_data(
        self,
        latitude: float,
        longitude: float,
        start_year: int = 1981,
        end_year: int = 2023
    ) -> Optional[Dict]:
        """
        Fetch monthly climate data for a specific location.

        Args:
            latitude: Latitude of the location
            longitude: Longitude of the location
            start_year: Start year for data collection
            end_year: End year for data collection

        Returns:
            Dictionary containing climate data or None if request fails
        """
        params = {
            "parameters": "T2M,RH2M,ALLSKY_SFC_SW_DWN",
            "community": "AG",
            "longitude": longitude,
            "latitude": latitude,
            "start": start_year,
            "end": end_year,
            "format": "JSON"
        }

        for attempt in range(self.max_retries):
            try:
                response = self.session.get(
                    self.BASE_URL,
                    params=params,
                    timeout=30
                )
                response.raise_for_status()
                return response.json()

            except requests.exceptions.Timeout:
                logger.warning(f"Timeout on attempt {attempt + 1}/{self.max_retries}")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)

            except requests.exceptions.RequestException as e:
                logger.error(f"Request failed: {e}")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    return None

        return None

    def parse_response(
        self,
        response: Dict,
        county: str,
        state_fp: str,
        county_fp: str,
        latitude: float,
        longitude: float
    ) -> List[Dict]:
        """
        Parse NASA POWER API response into structured records.

        Args:
            response: JSON response from API
            county: County name
            state_fp: State FIPS code
            county_fp: County FIPS code
            latitude: County latitude
            longitude: County longitude

        Returns:
            List of dictionaries with parsed data
        """
        records = []

        try:
            parameters = response.get("properties", {}).get("parameter", {})

            # Get the data for each parameter
            t2m_data = parameters.get("T2M", {})
            rh2m_data = parameters.get("RH2M", {})
            solar_data = parameters.get("ALLSKY_SFC_SW_DWN", {})

            # Iterate through dates
            for date_str in t2m_data.keys():
                record = {
                    "county": county,
                    "state_fp": state_fp,
                    "county_fp": county_fp,
                    "latitude": latitude,
                    "longitude": longitude,
                    "date": date_str,
                    "T2M": t2m_data.get(date_str),
                    "RH2M": rh2m_data.get(date_str),
                    "ALLSKY_SFC_SW_DWN": solar_data.get(date_str)
                }
                records.append(record)

        except Exception as e:
            logger.error(f"Error parsing response for {county}, {state_fp}: {e}")

        return records

    def fetch_all_counties(
        self,
        shapefile_path: str,
        output_file: str = "climate_data.csv",
        start_year: int = 1981,
        end_year: int = 2023,
        delay_between_requests: float = 0.5
    ) -> pd.DataFrame:
        """
        Fetch climate data for all US counties.

        Args:
            shapefile_path: Path to county shapefile
            output_file: Path to output CSV file
            start_year: Start year for data collection
            end_year: End year for data collection
            delay_between_requests: Delay in seconds between API requests

        Returns:
            DataFrame with all collected climate data
        """
        counties = self.get_us_county_centroids(shapefile_path)
        all_records = []

        total_counties = len(counties)
        logger.info(f"Starting data collection for {total_counties} counties")

        for idx, row in counties.iterrows():
            county = row["county"]
            state_fp = row["state_fp"]
            county_fp = row["county_fp"]
            lat = row["latitude"]
            lon = row["longitude"]

            logger.info(f"Processing {idx + 1}/{total_counties}: {county} (State: {state_fp}, County: {county_fp})")

            # Fetch data
            response = self.fetch_climate_data(lat, lon, start_year, end_year)

            if response:
                records = self.parse_response(response, county, state_fp, county_fp, lat, lon)
                all_records.extend(records)
                logger.info(f"  -> Collected {len(records)} monthly records")
            else:
                logger.error(f"  -> Failed to fetch data for {county}")

            # Rate limiting to avoid overwhelming the API
            time.sleep(delay_between_requests)

        # Create DataFrame
        df = pd.DataFrame(all_records)

        # Save to CSV
        df.to_csv(output_file, index=False)
        logger.info(f"Data saved to {output_file}")
        logger.info(f"Total records: {len(df)}")

        return df


def main():
    """Main execution function."""
    # Initialize fetcher
    fetcher = NASAPowerDataFetcher(max_retries=3, retry_delay=5)

    shapefile_path = "/content/drive/MyDrive/shapefiles/tl_2018_us_county.shp"

    # Fetch data for all counties (2005-2023)
    df = fetcher.fetch_all_counties(
        shapefile_path=shapefile_path,
        output_file="us_county_climate_data.csv",
        start_year=2005,
        end_year=2023,
        delay_between_requests=0.5
    )

    # Display summary statistics
    print("\n=== Data Collection Summary ===")
    print(f"Total records: {len(df)}")
    print(f"Counties: {df['county'].nunique()}")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    print("\nSample data:")
    print(df.head())

    print("\nMissing values:")
    print(df.isnull().sum())


if __name__ == "__main__":
    main()

2025-11-29 08:22:42,473 - INFO - Loading county centroids from /content/drive/MyDrive/shapefiles/tl_2018_us_county.shp


/tmp/ipython-input-1574355303.py:51: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["latitude"] = counties.geometry.centroid.y
/tmp/ipython-input-1574355303.py:52: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["longitude"] = counties.geometry.centroid.x


Streaming output truncated to the last 5000 lines.
2025-11-29 10:09:56,472 - INFO -   -> Collected 247 monthly records
2025-11-29 10:09:56,974 - INFO - Processing 752/3233: Sherman (State: 20, County: 181)
2025-11-29 10:10:04,633 - INFO -   -> Collected 247 monthly records
2025-11-29 10:10:05,135 - INFO - Processing 753/3233: Richland (State: 39, County: 139)
2025-11-29 10:10:12,812 - INFO -   -> Collected 247 monthly records
2025-11-29 10:10:13,313 - INFO - Processing 754/3233: Adams (State: 17, County: 001)
2025-11-29 10:10:20,830 - INFO -   -> Collected 247 monthly records
2025-11-29 10:10:21,332 - INFO - Processing 755/3233: Champaign (State: 39, County: 021)
2025-11-29 10:10:28,941 - INFO -   -> Collected 247 monthly records
2025-11-29 10:10:29,442 - INFO - Processing 756/3233: Sullivan (State: 33, County: 019)
2025-11-29 10:10:37,338 - INFO -   -> Collected 247 monthly records
2025-11-29 10:10:37,839 - INFO - Processing 757/3233: Iroquois (State: 17, County: 075)
2025-11-29 10:10

In [ ]:
import logging
import sys

# Force logging to display immediately
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout,
    force=True
)

In [ ]:
# Quick test
import requests
response = requests.get("https://power.larc.nasa.gov/api/temporal/monthly/point?parameters=T2M&longitude=-118&latitude=34&start=2023&end=2023&format=JSON")
print(response.status_code)

422


In [ ]:
import ee
ee.Authenticate()

True

In [ ]:
# This creates a new Earth Engine project
!earthengine create project crop-yield-project

E0000 00:00:1764485443.371087    3229 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764485443.378484    3229 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764485443.400447    3229 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764485443.400495    3229 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764485443.400500    3229 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764485443.400505    3229 computation_placer.cc:177] computation placer already registered. Please check linka

In [ ]:
!earthengine set_project crop-yield-analysis-479806

E0000 00:00:1764485957.885319    5304 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764485957.902376    5304 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764485957.950003    5304 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764485957.950084    5304 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764485957.950090    5304 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764485957.950095    5304 computation_placer.cc:177] computation placer already registered. Please check linka

In [ ]:
import ee
import pandas as pd
import geopandas as gpd
from datetime import datetime
import time
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class MODISDataExtractor:
    """Extract MODIS vegetation indices for US counties using Google Earth Engine."""

    def __init__(self, project=None):
        """
        Initialize Earth Engine and authenticate.

        Args:
            project: Google Cloud project ID (e.g., 'ee-your-project')
        """
        try:
            if project:
                ee.Initialize(project=project)
            else:
                ee.Initialize()
            logger.info("Earth Engine initialized successfully")
        except Exception as e:
            logger.info("Authenticating Earth Engine...")
            logger.info("Please follow the authentication prompts...")
            ee.Authenticate()

            # Try to initialize again with project
            if project:
                ee.Initialize(project=project)
            else:
                # If no project specified, Earth Engine will prompt for one
                logger.info("No project specified. Please provide a project ID.")
                logger.info("You can create one at: https://console.cloud.google.com/")
                raise Exception("Project ID required. Use: MODISDataExtractor(project='your-project-id')")

            logger.info("Earth Engine authenticated and initialized")

    def load_counties(self, shapefile_path: str) -> gpd.GeoDataFrame:
        """
        Load county shapefile.

        Args:
            shapefile_path: Path to county shapefile

        Returns:
            GeoDataFrame with county boundaries
        """
        logger.info(f"Loading counties from {shapefile_path}")
        counties = gpd.read_file(shapefile_path)

        # Ensure WGS84 projection for Earth Engine
        if counties.crs.to_epsg() != 4326:
            counties = counties.to_crs('EPSG:4326')

        logger.info(f"Loaded {len(counties)} counties")
        return counties

    def county_to_ee_geometry(self, geometry):
        """
        Convert GeoPandas geometry to Earth Engine geometry.

        Args:
            geometry: GeoPandas geometry object

        Returns:
            Earth Engine Geometry
        """
        # Convert to GeoJSON format
        geojson = geometry.__geo_interface__
        return ee.Geometry(geojson)

    def extract_modis_data(
        self,
        geometry,
        start_date: str,
        end_date: str
    ) -> pd.DataFrame:
        """
        Extract monthly MODIS data for a specific geometry.

        Args:
            geometry: Earth Engine geometry
            start_date: Start date (YYYY-MM-DD)
            end_date: End date (YYYY-MM-DD)

        Returns:
            DataFrame with monthly MODIS data
        """

        modis = ee.ImageCollection('MODIS/061/MOD13A2') \
            .filterDate(start_date, end_date) \
            .filterBounds(geometry)

        # Function to extract values for each image
        def extract_values(image):
            # Get the date
            date = ee.Date(image.get('system:time_start'))

            # Calculate mean values over the county
            stats = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geometry,
                scale=1000,  # 1km resolution
                maxPixels=1e9
            )

            # Return feature with properties
            return ee.Feature(None, {
                'date': date.format('YYYY-MM-dd'),
                'NDVI': stats.get('NDVI'),
                'EVI': stats.get('EVI'),
                'sur_refl_b01': stats.get('sur_refl_b01'),  # Red band
                'sur_refl_b02': stats.get('sur_refl_b02'),  # NIR band
            })

        # Map over collection
        features = modis.map(extract_values)

        # Convert to Python list
        try:
            feature_list = features.getInfo()['features']

            # Extract properties
            records = []
            for feature in feature_list:
                props = feature['properties']
                if props.get('NDVI') is not None:  # Skip if no data
                    records.append({
                        'date': props['date'],
                        'NDVI': props['NDVI'] / 10000.0,  # Scale factor
                        'EVI': props['EVI'] / 10000.0,
                        'red': props.get('sur_refl_b01', 0) / 10000.0,
                        'nir': props.get('sur_refl_b02', 0) / 10000.0,
                    })

            return pd.DataFrame(records)

        except Exception as e:
            logger.error(f"Error extracting data: {e}")
            return pd.DataFrame()

    def aggregate_to_monthly(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Aggregate 16-day MODIS data to monthly averages.

        Args:
            df: DataFrame with MODIS data

        Returns:
            DataFrame with monthly aggregated data
        """
        if df.empty:
            return df

        # Parse date
        df['date'] = pd.to_datetime(df['date'])
        df['year_month'] = df['date'].dt.to_period('M')

        # Aggregate to monthly
        monthly = df.groupby('year_month').agg({
            'NDVI': 'mean',
            'EVI': 'mean',
            'red': 'mean',
            'nir': 'mean'
        }).reset_index()

        # Format date as YYYYMM
        monthly['date'] = monthly['year_month'].astype(str).str.replace('-', '')
        monthly = monthly.drop('year_month', axis=1)

        # Calculate NDWI (water stress indicator)
        # NDWI = (NIR - Red) / (NIR + Red)
        monthly['NDWI'] = (monthly['nir'] - monthly['red']) / (monthly['nir'] + monthly['red'] + 1e-10)

        return monthly[['date', 'NDVI', 'EVI', 'NDWI']]

    def extract_all_counties(
        self,
        shapefile_path: str,
        output_file: str = "county_modis_data.csv",
        start_year: int = 2005,
        end_year: int = 2023
    ) -> pd.DataFrame:
        """
        Extract MODIS data for all counties.

        Args:
            shapefile_path: Path to county shapefile
            output_file: Output CSV file path
            start_year: Start year for data extraction
            end_year: End year for data extraction

        Returns:
            DataFrame with all MODIS data
        """
        # Load counties
        counties = self.load_counties(shapefile_path)

        # Date range
        start_date = f"{start_year}-01-01"
        end_date = f"{end_year}-12-31"

        all_records = []
        total_counties = len(counties)

        logger.info(f"Starting MODIS extraction for {total_counties} counties")
        logger.info(f"Date range: {start_date} to {end_date}")

        for idx, row in counties.iterrows():
            county_name = row['NAME']
            state_fp = row['STATEFP']
            county_fp = row['COUNTYFP']

            logger.info(f"Processing {idx + 1}/{total_counties}: {county_name} (State: {state_fp}, County: {county_fp})")

            try:
                # Convert geometry to Earth Engine format
                ee_geometry = self.county_to_ee_geometry(row.geometry)

                # Extract MODIS data
                modis_data = self.extract_modis_data(
                    ee_geometry,
                    start_date,
                    end_date
                )

                if not modis_data.empty:
                    # Aggregate to monthly
                    monthly_data = self.aggregate_to_monthly(modis_data)

                    # Add county information
                    monthly_data['county'] = county_name
                    monthly_data['state_fp'] = state_fp
                    monthly_data['county_fp'] = county_fp

                    # Get centroid for lat/lon
                    centroid = row.geometry.centroid
                    monthly_data['latitude'] = centroid.y
                    monthly_data['longitude'] = centroid.x

                    all_records.append(monthly_data)
                    logger.info(f"  -> Collected {len(monthly_data)} monthly records")
                else:
                    logger.warning(f"  -> No data available for {county_name}")

                # Small delay to avoid overwhelming Earth Engine
                time.sleep(0.1)

            except Exception as e:
                logger.error(f"  -> Error processing {county_name}: {e}")
                continue

        # Combine all data
        if all_records:
            df = pd.concat(all_records, ignore_index=True)

            # Reorder columns
            df = df[['county', 'state_fp', 'county_fp', 'latitude', 'longitude',
                     'date', 'NDVI', 'EVI', 'NDWI']]

            # Save to CSV
            df.to_csv(output_file, index=False)
            logger.info(f"Data saved to {output_file}")
            logger.info(f"Total records: {len(df)}")

            return df
        else:
            logger.error("No data collected!")
            return pd.DataFrame()


def main():
    """Main execution function."""



    project_id = 'crop-yield-analysis-479806'

    extractor = MODISDataExtractor(project=project_id)

    shapefile_path = "/content/drive/MyDrive/shapefiles/tl_2018_us_county.shp"

    df = extractor.extract_all_counties(
        shapefile_path=shapefile_path,
        output_file="us_county_modis_data.csv",
        start_year=2005,
        end_year=2023
    )

    if not df.empty:
        print("\n=== MODIS Data Collection Summary ===")
        print(f"Total records: {len(df)}")
        print(f"Counties: {df['county'].nunique()}")
        print(f"Date range: {df['date'].min()} to {df['date'].max()}")
        print("\nSample data:")
        print(df.head(10))
        print("\nMissing values:")
        print(df.isnull().sum())
        print("\nNDVI statistics:")
        print(df['NDVI'].describe())


if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
2025-11-30 07:59:58,628 - INFO - Processing 765/3233: McCurtain (State: 40, County: 089)
2025-11-30 08:00:06,339 - INFO -   -> Collected 228 monthly records
2025-11-30 08:00:06,441 - INFO - Processing 766/3233: Tift (State: 13, County: 277)
2025-11-30 08:00:09,104 - INFO -   -> Collected 228 monthly records
2025-11-30 08:00:09,206 - INFO - Processing 767/3233: Cass (State: 31, County: 025)
2025-11-30 08:00:11,522 - INFO -   -> Collected 228 monthly records
2025-11-30 08:00:11,623 - INFO - Processing 768/3233: Kingsbury (State: 46, County: 077)
2025-11-30 08:00:18,630 - INFO -   -> Collected 228 monthly records
2025-11-30 08:00:18,732 - INFO - Processing 769/3233: Converse (State: 56, County: 009)
2025-11-30 08:00:23,052 - INFO -   -> Collected 228 monthly records
2025-11-30 08:00:23,153 - INFO - Processing 770/3233: Herkimer (State: 36, County: 043)
2025-11-30 08:00:26,057 - INFO -   -> Collected 228 monthly records
2025-11-30 08:00:26

In [ ]:

import pandas as pd

corn_yield_data = pd.read_csv('/content/drive/MyDrive/project files/corn_yield_data.csv')
soybeans_yield_data = pd.read_csv('/content/drive/MyDrive/project files/soybeans_yield_data.csv')


print(corn_yield_data.head())
print(soybeans_yield_data.head())

  Program  Year Period  Week Ending Geo Level    State  State ANSI  \
0  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
1  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
2  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
3  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
4  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   

  Ag District  Ag District Code          County  ...  Zip Code  Region  \
0         NaN                99  OTHER COUNTIES  ...       NaN     NaN   
1  BLACK BELT                40          DALLAS  ...       NaN     NaN   
2  BLACK BELT                40          ELMORE  ...       NaN     NaN   
3  BLACK BELT                40           MACON  ...       NaN     NaN   
4  BLACK BELT                40         MARENGO  ...       NaN     NaN   

   watershed_code  Watershed  Commodity  \
0               0        NaN       CORN   
1               0        NaN       CORN   
2    

In [ ]:
# Show me the exact column structure
print("Corn columns:", corn_yield_data.columns.tolist())
print("Soybean columns:", soybeans_yield_data.columns.tolist())

# Check the data types and structure
print("\nCorn info:")
print(corn_yield_data.info())

print("\nSoybean info:")
print(soybeans_yield_data.info())

# Check year range
print("\nCorn years:", corn_yield_data['Year'].min(), "to", corn_yield_data['Year'].max())
print("Soybean years:", soybeans_yield_data['Year'].min(), "to", soybeans_yield_data['Year'].max())

Corn columns: ['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed', 'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value', 'CV (%)']
Soybean columns: ['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed', 'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value', 'CV (%)']

Corn info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32654 entries, 0 to 32653
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Program           32654 non-null  object 
 1   Year              32654 non-null  int64  
 2   Period            32654 non-null  object 
 3   Week Ending       0 non-null      float64
 4   Geo Level         32654 n

In [ ]:
import pandas as pd
import numpy as np
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class CropYieldDataMerger:
    """Merge climate, satellite, and yield data for crop yield volatility analysis."""

    def __init__(self):
        self.climate_data = None
        self.satellite_data = None
        self.yield_data = None
        self.merged_data = None

    def load_data(
        self,
        climate_path: str,
        satellite_path: str,
        corn_yield_path: str,
        soybean_yield_path: str
    ):
        """Load all datasets."""
        logger.info("Loading datasets...")

        # Load climate data
        self.climate_data = pd.read_csv(climate_path)
        logger.info(f"Loaded climate data: {len(self.climate_data)} records")

        # Load satellite data
        self.satellite_data = pd.read_csv(satellite_path)
        logger.info(f"Loaded satellite data: {len(self.satellite_data)} records")

        # Load and combine yield data
        corn = pd.read_csv(corn_yield_path)
        soybean = pd.read_csv(soybean_yield_path)

        # Add crop identifier
        corn['crop'] = 'corn'
        soybean['crop'] = 'soybean'

        # Combine both crops
        self.yield_data = pd.concat([corn, soybean], ignore_index=True)
        logger.info(f"Loaded yield data: {len(self.yield_data)} records (corn: {len(corn)}, soybean: {len(soybean)})")

        return self

    def prepare_climate_data(self):
        """Aggregate climate data to growing season features."""
        logger.info("Preparing climate data...")

        # Standardize FIPS codes to strings
        self.climate_data['state_fp'] = self.climate_data['state_fp'].astype(str).str.zfill(2)
        self.climate_data['county_fp'] = self.climate_data['county_fp'].astype(str).str.zfill(3)

        # Parse date and extract year/month
        self.climate_data['date_str'] = self.climate_data['date'].astype(str)
        self.climate_data['year'] = self.climate_data['date_str'].str[:4].astype(int)
        self.climate_data['month'] = self.climate_data['date_str'].str[4:6].astype(int)

        # Filter to growing season (April-October)
        growing_season = self.climate_data[
            (self.climate_data['month'] >= 4) &
            (self.climate_data['month'] <= 10)
        ].copy()

        logger.info(f"Filtered to growing season: {len(growing_season)} records")

        # Aggregate by county and year
        climate_features = growing_season.groupby(
            ['state_fp', 'county_fp', 'year']
        ).agg({
            'T2M': ['mean', 'max', 'min', 'std'],
            'RH2M': ['mean', 'std'],
            'ALLSKY_SFC_SW_DWN': ['mean', 'std'],
            'latitude': 'first',
            'longitude': 'first',
            'county': 'first'
        }).reset_index()

        # Flatten column names
        climate_features.columns = ['_'.join(col).strip('_') for col in climate_features.columns.values]

        # Add extreme temperature features
        extreme_heat = growing_season[growing_season['T2M'] > 30].groupby(
            ['state_fp', 'county_fp', 'year']
        ).size().reset_index(name='extreme_heat_days')

        climate_features = climate_features.merge(
            extreme_heat,
            on=['state_fp', 'county_fp', 'year'],
            how='left'
        )
        climate_features['extreme_heat_days'] = climate_features['extreme_heat_days'].fillna(0)

        logger.info(f"Created climate features: {len(climate_features)} county-year combinations")

        return climate_features

    def prepare_satellite_data(self):
        """Aggregate satellite data to growing season features."""
        logger.info("Preparing satellite data...")

        # Standardize FIPS codes to strings
        self.satellite_data['state_fp'] = self.satellite_data['state_fp'].astype(str).str.zfill(2)
        self.satellite_data['county_fp'] = self.satellite_data['county_fp'].astype(str).str.zfill(3)

        # Parse date and extract year/month
        self.satellite_data['date_str'] = self.satellite_data['date'].astype(str)
        self.satellite_data['year'] = self.satellite_data['date_str'].str[:4].astype(int)
        self.satellite_data['month'] = self.satellite_data['date_str'].str[4:6].astype(int)

        # Filter to growing season (April-October)
        growing_season = self.satellite_data[
            (self.satellite_data['month'] >= 4) &
            (self.satellite_data['month'] <= 10)
        ].copy()

        logger.info(f"Filtered to growing season: {len(growing_season)} records")

        # Aggregate by county and year
        satellite_features = growing_season.groupby(
            ['state_fp', 'county_fp', 'year']
        ).agg({
            'NDVI': ['mean', 'max', 'min', 'std'],
            'EVI': ['mean', 'std'],
            'NDWI': ['mean', 'std']
        }).reset_index()

        # Flatten column names
        satellite_features.columns = ['_'.join(col).strip('_') for col in satellite_features.columns.values]

        logger.info(f"Created satellite features: {len(satellite_features)} county-year combinations")

        return satellite_features

    def prepare_yield_data(self):
        """Clean and prepare yield data."""
        logger.info("Preparing yield data...")

        # Keep only county-level data
        yield_clean = self.yield_data[
            self.yield_data['Geo Level'] == 'COUNTY'
        ].copy()
        yield_clean['County ANSI'] = yield_clean['County ANSI'].fillna(0).astype(int)
        yield_clean['county_fp'] = yield_clean['County ANSI'].astype(str).str.zfill(3)
        yield_clean['state_fp'] = yield_clean['State ANSI'].astype(str).str.zfill(2)

        # Remove rows with missing FIPS codes
        yield_clean = yield_clean[
            (yield_clean['County ANSI'] != 0) &
            (yield_clean['county_fp'] != '000')
        ]

        # Select relevant columns
        yield_clean = yield_clean[[
            'Year', 'state_fp', 'county_fp', 'State', 'County',
            'crop', 'Value', 'CV (%)'
        ]].copy()

        # Rename columns
        yield_clean.columns = [
            'year', 'state_fp', 'county_fp', 'state_name', 'county_name',
            'crop', 'yield_value', 'yield_cv'
        ]

        logger.info(f"Cleaned yield data: {len(yield_clean)} records")
        logger.info(f"  Corn records: {len(yield_clean[yield_clean['crop'] == 'corn'])}")
        logger.info(f"  Soybean records: {len(yield_clean[yield_clean['crop'] == 'soybean'])}")

        return yield_clean

    def merge_datasets(self):
        """Merge all datasets together."""
        logger.info("Merging all datasets...")

        # Prepare each dataset
        climate_features = self.prepare_climate_data()
        satellite_features = self.prepare_satellite_data()
        yield_clean = self.prepare_yield_data()

        # Merge climate and satellite
        logger.info("Merging climate and satellite data...")
        climate_satellite = climate_features.merge(
            satellite_features,
            on=['state_fp', 'county_fp', 'year'],
            how='inner'
        )
        logger.info(f"Climate + Satellite merged: {len(climate_satellite)} records")

        # Merge with yield data
        logger.info("Merging with yield data...")
        final_data = climate_satellite.merge(
            yield_clean,
            on=['state_fp', 'county_fp', 'year'],
            how='inner'
        )

        logger.info(f"Final merged dataset: {len(final_data)} records")
        logger.info(f"  Unique counties: {final_data['county_fp'].nunique()}")
        logger.info(f"  Year range: {final_data['year'].min()} to {final_data['year'].max()}")
        logger.info(f"  Corn records: {len(final_data[final_data['crop'] == 'corn'])}")
        logger.info(f"  Soybean records: {len(final_data[final_data['crop'] == 'soybean'])}")

        self.merged_data = final_data
        return final_data

    def save_merged_data(self, output_path: str):
        """Save merged dataset to CSV."""
        if self.merged_data is None:
            raise ValueError("No merged data to save. Run merge_datasets() first.")

        self.merged_data.to_csv(output_path, index=False)
        logger.info(f"Saved merged data to {output_path}")

    def get_summary_statistics(self):
        """Print summary statistics of merged data."""
        if self.merged_data is None:
            raise ValueError("No merged data. Run merge_datasets() first.")

        print("\n" + "="*60)
        print("MERGED DATASET SUMMARY")
        print("="*60)

        print(f"\nTotal records: {len(self.merged_data):,}")
        print(f"Unique counties: {self.merged_data['county_fp'].nunique()}")
        print(f"Year range: {self.merged_data['year'].min()} - {self.merged_data['year'].max()}")

        print("\n--- By Crop ---")
        print(self.merged_data['crop'].value_counts())

        print("\n--- By State (Top 10) ---")
        print(self.merged_data['state_name'].value_counts().head(10))

        print("\n--- Yield Statistics ---")
        print(self.merged_data.groupby('crop')['yield_value'].describe())

        print("\n--- Climate Statistics ---")
        print(self.merged_data[['T2M_mean', 'RH2M_mean', 'extreme_heat_days']].describe())

        print("\n--- Satellite Statistics ---")
        print(self.merged_data[['NDVI_mean', 'EVI_mean', 'NDWI_mean']].describe())

        print("\n--- Missing Values ---")
        print(self.merged_data.isnull().sum()[self.merged_data.isnull().sum() > 0])

        print("\n--- Sample Data ---")
        print(self.merged_data.head(10))

        return self.merged_data


def main():
    """Main execution function."""

    # Initialize merger
    merger = CropYieldDataMerger()

    # Load all data
    merger.load_data(
        climate_path='/content/drive/MyDrive/project files/us_county_climate_data.csv',
        satellite_path='/content/drive/MyDrive/project files/us_county_modis_data.csv',
        corn_yield_path='/content/drive/MyDrive/project files/corn_yield_data.csv',
        soybean_yield_path='/content/drive/MyDrive/project files/soybeans_yield_data.csv'
    )

    # Merge datasets
    merged_data = merger.merge_datasets()

    # Save merged data
    merger.save_merged_data('merged_crop_climate_data.csv')

    # Print summary
    merger.get_summary_statistics()

    return merged_data


if __name__ == "__main__":
    merged_data = main()


MERGED DATASET SUMMARY

Total records: 56,474
Unique counties: 196
Year range: 2005 - 2023

--- By Crop ---
crop
corn       30174
soybean    26300
Name: count, dtype: int64

--- By State (Top 10) ---
state_name
IOWA              3624
ILLINOIS          3567
INDIANA           3169
OHIO              2925
MISSOURI          2874
KENTUCKY          2854
KANSAS            2784
NEBRASKA          2767
MINNESOTA         2712
NORTH CAROLINA    2637
Name: count, dtype: int64

--- Yield Statistics ---
           count        mean        std  min    25%    50%    75%    max
crop                                                                    
corn     30174.0  142.519414  40.698517  0.0  117.0  146.9  172.3  277.1
soybean  26300.0   43.057593  11.373263  0.7   35.4   44.0   51.2   80.4

--- Climate Statistics ---
           T2M_mean     RH2M_mean  extreme_heat_days
count  56474.000000  56474.000000       56474.000000
mean      19.285933     71.298075           0.060842
std        2.973903      7.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class VolatilityAnalyzer:
    """Analyze crop yield volatility and climate trends."""

    def __init__(self, data_path: str):
        """Load merged dataset."""
        logger.info(f"Loading data from {data_path}")
        self.data = pd.read_csv(data_path)
        logger.info(f"Loaded {len(self.data)} records")

        # Define periods for comparison
        self.early_start = 2005
        self.early_end = 2014
        self.late_start = 2015
        self.late_end = 2023

        self.volatility_data = None
        self.climate_trends = None
        self.final_analysis = None

    def calculate_yield_volatility(self):
        """Calculate yield volatility metrics for each county-crop combination."""
        logger.info("Calculating yield volatility metrics...")

        # Group by county and crop
        volatility_metrics = []

        for (state_fp, county_fp, crop), group in self.data.groupby(['state_fp', 'county_fp', 'crop']):
            # Skip if too few years
            if len(group) < 5:
                continue

            # Get county info
            county_name = group['county_name'].iloc[0]
            state_name = group['state_name'].iloc[0]
            lat = group['latitude'].iloc[0] if 'latitude' in group.columns else np.nan
            lon = group['longitude'].iloc[0] if 'longitude' in group.columns else np.nan

            # Overall statistics
            yields = group['yield_value']

            # Calculate volatility metrics
            metrics = {
                'state_fp': state_fp,
                'county_fp': county_fp,
                'crop': crop,
                'county_name': county_name,
                'state_name': state_name,

                # Overall metrics
                'yield_mean': yields.mean(),
                'yield_std': yields.std(),
                'yield_cv': (yields.std() / yields.mean()) * 100,  # Coefficient of variation (%)
                'yield_min': yields.min(),
                'yield_max': yields.max(),
                'yield_range': yields.max() - yields.min(),
                'n_years': len(group),

                # Trend analysis
                'yield_trend_slope': self._calculate_trend(group['year'], yields)
            }

            # Period comparison (early vs late)
            early_data = group[group['year'] <= self.early_end]
            late_data = group[group['year'] >= self.late_start]

            if len(early_data) >= 3 and len(late_data) >= 3:
                early_yields = early_data['yield_value']
                late_yields = late_data['yield_value']

                metrics.update({
                    # Early period (2005-2014)
                    'early_yield_mean': early_yields.mean(),
                    'early_yield_std': early_yields.std(),
                    'early_yield_cv': (early_yields.std() / early_yields.mean()) * 100,
                    'early_n_years': len(early_data),

                    # Late period (2015-2023)
                    'late_yield_mean': late_yields.mean(),
                    'late_yield_std': late_yields.std(),
                    'late_yield_cv': (late_yields.std() / late_yields.mean()) * 100,
                    'late_n_years': len(late_data),

                    # Changes (late - early)
                    'yield_mean_change': late_yields.mean() - early_yields.mean(),
                    'yield_std_change': late_yields.std() - early_yields.std(),
                    'yield_cv_change': ((late_yields.std() / late_yields.mean()) -
                                       (early_yields.std() / early_yields.mean())) * 100,

                    # Percent changes
                    'yield_mean_pct_change': ((late_yields.mean() - early_yields.mean()) /
                                             early_yields.mean()) * 100,
                    'yield_std_pct_change': ((late_yields.std() - early_yields.std()) /
                                            early_yields.std()) * 100 if early_yields.std() > 0 else 0,
                })

                # Risk classification
                if metrics['yield_cv_change'] > 5:
                    metrics['risk_category'] = 'High Risk (Increasing)'
                elif metrics['yield_cv_change'] > 0:
                    metrics['risk_category'] = 'Medium Risk (Slight Increase)'
                elif metrics['yield_cv_change'] > -5:
                    metrics['risk_category'] = 'Low Risk (Stable)'
                else:
                    metrics['risk_category'] = 'Improving (Decreasing)'
            else:
                # Not enough data for period comparison
                metrics.update({
                    'early_yield_mean': np.nan,
                    'early_yield_std': np.nan,
                    'early_yield_cv': np.nan,
                    'early_n_years': len(early_data),
                    'late_yield_mean': np.nan,
                    'late_yield_std': np.nan,
                    'late_yield_cv': np.nan,
                    'late_n_years': len(late_data),
                    'yield_mean_change': np.nan,
                    'yield_std_change': np.nan,
                    'yield_cv_change': np.nan,
                    'yield_mean_pct_change': np.nan,
                    'yield_std_pct_change': np.nan,
                    'risk_category': 'Insufficient Data'
                })

            volatility_metrics.append(metrics)

        self.volatility_data = pd.DataFrame(volatility_metrics)
        logger.info(f"Calculated volatility for {len(self.volatility_data)} county-crop combinations")

        return self.volatility_data

    def calculate_climate_trends(self):
        """Calculate climate and satellite trends for each county."""
        logger.info("Calculating climate trends...")

        climate_trends = []

        for (state_fp, county_fp), group in self.data.groupby(['state_fp', 'county_fp']):
            # Get county info
            county_name = group['county_name'].iloc[0]
            state_name = group['state_name'].iloc[0]

            # Period comparison
            early_data = group[group['year'] <= self.early_end]
            late_data = group[group['year'] >= self.late_start]

            if len(early_data) < 3 or len(late_data) < 3:
                continue

            trends = {
                'state_fp': state_fp,
                'county_fp': county_fp,
                'county_name': county_name,
                'state_name': state_name,
            }

            # Climate variables
            climate_vars = ['T2M_mean', 'T2M_max', 'T2M_std', 'extreme_heat_days',
                          'RH2M_mean', 'ALLSKY_SFC_SW_DWN_mean']

            for var in climate_vars:
                early_mean = early_data[var].mean()
                late_mean = late_data[var].mean()
                change = late_mean - early_mean
                pct_change = (change / early_mean * 100) if early_mean != 0 else 0

                trends[f'{var}_early'] = early_mean
                trends[f'{var}_late'] = late_mean
                trends[f'{var}_change'] = change
                trends[f'{var}_pct_change'] = pct_change

            # Satellite variables
            satellite_vars = ['NDVI_mean', 'NDVI_std', 'EVI_mean', 'NDWI_mean']

            for var in satellite_vars:
                early_mean = early_data[var].mean()
                late_mean = late_data[var].mean()
                change = late_mean - early_mean
                pct_change = (change / early_mean * 100) if early_mean != 0 else 0

                trends[f'{var}_early'] = early_mean
                trends[f'{var}_late'] = late_mean
                trends[f'{var}_change'] = change
                trends[f'{var}_pct_change'] = pct_change

            climate_trends.append(trends)

        self.climate_trends = pd.DataFrame(climate_trends)
        logger.info(f"Calculated climate trends for {len(self.climate_trends)} counties")

        return self.climate_trends

    def merge_volatility_and_trends(self):
        """Merge volatility metrics with climate trends."""
        logger.info("Merging volatility and climate trends...")

        if self.volatility_data is None:
            self.calculate_yield_volatility()
        if self.climate_trends is None:
            self.calculate_climate_trends()

        # Merge on county (not crop-specific for climate)
        self.final_analysis = self.volatility_data.merge(
            self.climate_trends,
            on=['state_fp', 'county_fp'],
            how='inner',
            suffixes=('', '_climate')
        )

        logger.info(f"Final analysis dataset: {len(self.final_analysis)} records")

        return self.final_analysis

    def _calculate_trend(self, x, y):
        """Calculate linear trend slope using least squares."""
        if len(x) < 3:
            return np.nan
        slope, _, _, _, _ = stats.linregress(x, y)
        return slope

    def identify_high_risk_counties(self, threshold_cv_change=3.0):
        """Identify counties with significant volatility increases."""
        if self.final_analysis is None:
            self.merge_volatility_and_trends()

        high_risk = self.final_analysis[
            self.final_analysis['yield_cv_change'] > threshold_cv_change
        ].copy()

        high_risk = high_risk.sort_values('yield_cv_change', ascending=False)

        logger.info(f"Identified {len(high_risk)} high-risk county-crop combinations")
        logger.info(f"  (CV change > {threshold_cv_change}%)")

        return high_risk

    def get_correlation_analysis(self):
        """Calculate correlations between climate trends and volatility changes."""
        if self.final_analysis is None:
            self.merge_volatility_and_trends()

        # Select relevant columns
        climate_change_cols = [col for col in self.final_analysis.columns if '_change' in col and 'yield' not in col]
        volatility_cols = ['yield_cv_change', 'yield_std_change']

        # Calculate correlations
        correlations = {}
        for vol_col in volatility_cols:
            corr_data = self.final_analysis[climate_change_cols + [vol_col]].corr()[vol_col]
            corr_data = corr_data.drop(vol_col).sort_values(ascending=False)
            correlations[vol_col] = corr_data

        return correlations

    def generate_summary_report(self, output_path='volatility_analysis_report.txt'):
        """Generate comprehensive summary report."""
        if self.final_analysis is None:
            self.merge_volatility_and_trends()

        with open(output_path, 'w') as f:
            f.write("="*80 + "\n")
            f.write("CROP YIELD VOLATILITY ANALYSIS REPORT\n")
            f.write("="*80 + "\n\n")

            # Overall statistics
            f.write("OVERALL STATISTICS\n")
            f.write("-"*80 + "\n")
            f.write(f"Total county-crop combinations analyzed: {len(self.volatility_data)}\n")
            f.write(f"Counties: {self.volatility_data['county_fp'].nunique()}\n")
            f.write(f"Crops: {', '.join(self.volatility_data['crop'].unique())}\n\n")

            # Volatility summary by crop
            f.write("VOLATILITY SUMMARY BY CROP\n")
            f.write("-"*80 + "\n")
            for crop in self.volatility_data['crop'].unique():
                crop_data = self.volatility_data[self.volatility_data['crop'] == crop]
                f.write(f"\n{crop.upper()}:\n")
                f.write(f"  Average Yield: {crop_data['yield_mean'].mean():.2f} bu/acre\n")
                f.write(f"  Average Volatility (CV): {crop_data['yield_cv'].mean():.2f}%\n")
                f.write(f"  Average CV Change (2015-2023 vs 2005-2014): {crop_data['yield_cv_change'].mean():.2f}%\n")

            # Risk categories
            f.write("\n\nRISK CATEGORIES\n")
            f.write("-"*80 + "\n")
            risk_counts = self.volatility_data['risk_category'].value_counts()
            for category, count in risk_counts.items():
                pct = (count / len(self.volatility_data)) * 100
                f.write(f"{category}: {count} ({pct:.1f}%)\n")

            # High-risk counties
            f.write("\n\nTOP 20 HIGH-RISK COUNTIES (Highest CV Increase)\n")
            f.write("-"*80 + "\n")
            high_risk = self.volatility_data.nlargest(20, 'yield_cv_change')
            for idx, row in high_risk.iterrows():
                f.write(f"{row['county_name']}, {row['state_name']} ({row['crop']}): "
                       f"CV change = +{row['yield_cv_change']:.2f}%\n")

            # Climate correlations
            f.write("\n\nCLIMATE CORRELATIONS WITH VOLATILITY CHANGE\n")
            f.write("-"*80 + "\n")
            correlations = self.get_correlation_analysis()
            f.write("\nTop 10 correlations with CV change:\n")
            for var, corr in correlations['yield_cv_change'].head(10).items():
                f.write(f"  {var}: {corr:.3f}\n")

        logger.info(f"Report saved to {output_path}")
        print(f"\nReport saved to: {output_path}")

        return output_path

    def save_results(self, prefix='volatility'):
        """Save all analysis results to CSV files."""
        logger.info("Saving results...")

        if self.volatility_data is not None:
            volatility_file = f'{prefix}_metrics.csv'
            self.volatility_data.to_csv(volatility_file, index=False)
            logger.info(f"Saved volatility metrics to {volatility_file}")

        if self.climate_trends is not None:
            climate_file = f'{prefix}_climate_trends.csv'
            self.climate_trends.to_csv(climate_file, index=False)
            logger.info(f"Saved climate trends to {climate_file}")

        if self.final_analysis is not None:
            final_file = f'{prefix}_final_analysis.csv'
            self.final_analysis.to_csv(final_file, index=False)
            logger.info(f"Saved final analysis to {final_file}")

        return self


def main():
    """Main execution function."""

    analyzer = VolatilityAnalyzer('merged_crop_climate_data.csv')

    # Calculate volatility metrics
    volatility_data = analyzer.calculate_yield_volatility()

    # Calculate climate trends
    climate_trends = analyzer.calculate_climate_trends()

    # Merge everything
    final_analysis = analyzer.merge_volatility_and_trends()

    high_risk = analyzer.identify_high_risk_counties(threshold_cv_change=3.0)

    correlations = analyzer.get_correlation_analysis()

    analyzer.save_results()

    analyzer.generate_summary_report()

    print("\n" + "="*80)
    print("VOLATILITY ANALYSIS SUMMARY")
    print("="*80)
    print(f"\nTotal county-crop combinations: {len(volatility_data)}")
    print(f"High-risk combinations (CV increase >3%): {len(high_risk)}")

    print("\n--- Risk Category Distribution ---")
    print(volatility_data['risk_category'].value_counts())

    print("\n--- Top 10 High-Risk Counties ---")
    print(high_risk[['county_name', 'state_name', 'crop', 'yield_cv_change']].head(10))

    print("\n--- Top Climate Correlations with Volatility Change ---")
    print(correlations['yield_cv_change'].head(10))

    print("\n--- Average Changes by Crop ---")
    print(volatility_data.groupby('crop')[['yield_mean_change', 'yield_cv_change']].mean())

    print("\n" + "="*80)
    print("Analysis complete! Check the generated files:")
    print("  - volatility_metrics.csv")
    print("  - volatility_climate_trends.csv")
    print("  - volatility_final_analysis.csv")
    print("  - volatility_analysis_report.txt")
    print("="*80)

    return analyzer, volatility_data, climate_trends, final_analysis, high_risk


if __name__ == "__main__":
    analyzer, volatility_data, climate_trends, final_analysis, high_risk = main()


Report saved to: volatility_analysis_report.txt

VOLATILITY ANALYSIS SUMMARY

Total county-crop combinations: 3574
High-risk combinations (CV increase >3%): 388

--- Risk Category Distribution ---
risk_category
Improving (Decreasing)           1616
Low Risk (Stable)                 851
Medium Risk (Slight Increase)     481
Insufficient Data                 386
High Risk (Increasing)            240
Name: count, dtype: int64

--- Top 10 High-Risk Counties ---
        county_name    state_name     crop  yield_cv_change
180          BARTOW       GEORGIA  soybean        54.020858
975         LINCOLN        KANSAS  soybean        44.812295
2338  GOLDEN VALLEY  NORTH DAKOTA     corn        31.822868
2580        CHOCTAW      OKLAHOMA  soybean        31.486499
3066       ATASCOSA         TEXAS     corn        26.387637
60           MONROE       ALABAMA  soybean        26.284673
3122          LAMAR         TEXAS  soybean        25.676433
1952         GARDEN      NEBRASKA     corn        25.6193

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


class VolatilityPredictor:
    """Predict crop yield volatility using climate and satellite data."""

    def __init__(self, data_path='volatility_final_analysis.csv'):
        """Load and prepare data."""
        print("="*80)
        print("CROP YIELD VOLATILITY PREDICTION MODEL")
        print("="*80)
        print("\n1. Loading data...")

        self.data = pd.read_csv(data_path)
        print(f"   Loaded {len(self.data)} records")

        self.models = {}
        self.results = {}
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None

    def prepare_features(self):
        """Prepare features and target for modeling."""
        print("\n2. Preparing features...")

        # Define feature columns
        climate_features = [
            'T2M_mean_change',
            'T2M_std_change',
            'T2M_max_change',
            'extreme_heat_days_change',
            'RH2M_mean_change',
            'ALLSKY_SFC_SW_DWN_mean_change'
        ]

        satellite_features = [
            'NDVI_mean_change',
            'NDVI_std_change',
            'EVI_mean_change',
            'NDWI_mean_change'
        ]

        baseline_features = [
            'early_yield_mean',
            'early_yield_cv'
        ]

        # Combine all features
        all_features = climate_features + satellite_features + baseline_features

        # Check for missing values
        clean_data = self.data[all_features + ['yield_cv_change', 'crop']].dropna()
        print(f"   Removed {len(self.data) - len(clean_data)} rows with missing values")
        print(f"   Final dataset: {len(clean_data)} samples")

        # Prepare features
        X = clean_data[all_features].copy()

        # Add crop type as dummy variable
        crop_dummies = pd.get_dummies(clean_data['crop'], prefix='crop', drop_first=True)
        X = pd.concat([X, crop_dummies], axis=1)

        # Target variable
        y = clean_data['yield_cv_change'].copy()

        self.feature_names = X.columns.tolist()

        print(f"   Features: {len(self.feature_names)}")
        print(f"   Target: yield_cv_change (volatility change %)")

        # Train-test split (80-20)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        print(f"   Training samples: {len(self.X_train)}")
        print(f"   Test samples: {len(self.X_test)}")

        return X, y

    def train_models(self):
        """Train all three models."""
        print("\n3. Training models...")
        print("-" * 80)

        # Model 1: Linear Regression (Baseline)
        print("\n   Model 1: Linear Regression (Baseline)")
        lr_model = LinearRegression()
        lr_model.fit(self.X_train, self.y_train)
        self.models['Linear Regression'] = lr_model
        self._evaluate_model('Linear Regression', lr_model)

        # Model 2: Random Forest (Primary)
        print("\n   Model 2: Random Forest (Primary Model)")
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
        rf_model.fit(self.X_train, self.y_train)
        self.models['Random Forest'] = rf_model
        self._evaluate_model('Random Forest', rf_model)

        # Model 3: XGBoost (Best Performance)
        print("\n   Model 3: XGBoost (Advanced Model)")
        xgb_model = XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            min_child_weight=3,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1
        )
        xgb_model.fit(self.X_train, self.y_train)
        self.models['XGBoost'] = xgb_model
        self._evaluate_model('XGBoost', xgb_model)

        print("\n" + "-" * 80)

    def _evaluate_model(self, name, model):
        """Evaluate a single model."""
        # Predictions
        y_train_pred = model.predict(self.X_train)
        y_test_pred = model.predict(self.X_test)

        # Metrics
        train_r2 = r2_score(self.y_train, y_train_pred)
        test_r2 = r2_score(self.y_test, y_test_pred)
        test_rmse = np.sqrt(mean_squared_error(self.y_test, y_test_pred))
        test_mae = mean_absolute_error(self.y_test, y_test_pred)

        # Cross-validation
        cv_scores = cross_val_score(model, self.X_train, self.y_train,
                                    cv=5, scoring='r2', n_jobs=-1)

        # Store results
        self.results[name] = {
            'train_r2': train_r2,
            'test_r2': test_r2,
            'test_rmse': test_rmse,
            'test_mae': test_mae,
            'cv_r2_mean': cv_scores.mean(),
            'cv_r2_std': cv_scores.std(),
            'predictions': y_test_pred
        }

        # Print results
        print(f"      Train R²: {train_r2:.4f}")
        print(f"      Test R²:  {test_r2:.4f}")
        print(f"      Test RMSE: {test_rmse:.4f}")
        print(f"      Test MAE:  {test_mae:.4f}")
        print(f"      CV R² (5-fold): {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

    def compare_models(self):
        """Compare all models side by side."""
        print("\n4. Model Comparison")
        print("-" * 80)

        comparison_df = pd.DataFrame({
            'Model': list(self.results.keys()),
            'Test R²': [r['test_r2'] for r in self.results.values()],
            'Test RMSE': [r['test_rmse'] for r in self.results.values()],
            'Test MAE': [r['test_mae'] for r in self.results.values()],
            'CV R² Mean': [r['cv_r2_mean'] for r in self.results.values()]
        }).sort_values('Test R²', ascending=False)

        print("\n" + comparison_df.to_string(index=False))

        best_model = comparison_df.iloc[0]['Model']
        print(f"\n   ✓ Best Model: {best_model}")

        return comparison_df

    def analyze_feature_importance(self):
        """Analyze feature importance from Random Forest and XGBoost."""
        print("\n5. Feature Importance Analysis")
        print("-" * 80)

        # Random Forest importance
        rf_importance = pd.DataFrame({
            'Feature': self.feature_names,
            'Importance': self.models['Random Forest'].feature_importances_
        }).sort_values('Importance', ascending=False)

        print("\n   Random Forest - Top 10 Features:")
        print(rf_importance.head(10).to_string(index=False))

        # XGBoost importance
        xgb_importance = pd.DataFrame({
            'Feature': self.feature_names,
            'Importance': self.models['XGBoost'].feature_importances_
        }).sort_values('Importance', ascending=False)

        print("\n   XGBoost - Top 10 Features:")
        print(xgb_importance.head(10).to_string(index=False))

        return rf_importance, xgb_importance

    def analyze_linear_coefficients(self):
        """Analyze coefficients from Linear Regression."""
        print("\n6. Linear Regression Coefficients")
        print("-" * 80)

        coefficients = pd.DataFrame({
            'Feature': self.feature_names,
            'Coefficient': self.models['Linear Regression'].coef_
        }).sort_values('Coefficient', key=abs, ascending=False)

        print("\n   Top 10 Most Influential Features:")
        print(coefficients.head(10).to_string(index=False))

        print("\n   Interpretation:")
        top_feature = coefficients.iloc[0]
        print(f"   - For every 1-unit increase in {top_feature['Feature']},")
        print(f"     yield volatility changes by {top_feature['Coefficient']:.3f}%")

        return coefficients

    def predict_high_risk_counties(self, threshold=5.0):
        """Identify counties predicted to become high-risk."""
        print(f"\n7. High-Risk County Predictions (CV change > {threshold}%)")
        print("-" * 80)

        # Use best model (Random Forest)
        best_model = self.models['Random Forest']

        # Prepare full dataset with same features as training
        # Get base features (without crop dummy)
        base_features = [f for f in self.feature_names if not f.startswith('crop_')]

        # Prepare data
        clean_data = self.data.dropna(subset=base_features + ['crop'])
        X_full = clean_data[base_features].copy()

        # Add crop dummy if needed
        if 'crop_soybean' in self.feature_names:
            crop_dummies = pd.get_dummies(clean_data['crop'], prefix='crop', drop_first=True)
            X_full = pd.concat([X_full, crop_dummies], axis=1)

        predictions = best_model.predict(X_full)

        # Add predictions to clean data
        result_data = clean_data.copy()
        result_data['predicted_cv_change'] = predictions
        result_data['predicted_high_risk'] = predictions > threshold

        # Identify high-risk counties
        high_risk = result_data[result_data['predicted_high_risk'] == True].copy()
        high_risk = high_risk.sort_values('predicted_cv_change', ascending=False)

        print(f"\n   Total counties predicted to be high-risk: {len(high_risk)}")
        print(f"\n   Top 20 Highest Risk Counties:")
        print(high_risk[['county_name', 'state_name', 'crop',
                        'predicted_cv_change', 'yield_cv_change']].head(20).to_string(index=False))

        # Update main data with predictions (for saving later)
        self.data = result_data

        return high_risk

    def create_visualizations(self):
        """Create comprehensive visualizations."""
        print("\n8. Creating visualizations...")

        # Figure 1: Model Comparison
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        # R² comparison
        models = list(self.results.keys())
        r2_scores = [self.results[m]['test_r2'] for m in models]

        axes[0].bar(models, r2_scores, color=['#3498db', '#2ecc71', '#e74c3c'])
        axes[0].set_ylabel('R² Score', fontsize=12)
        axes[0].set_title('Model Performance Comparison (R²)', fontsize=14, fontweight='bold')
        axes[0].set_ylim(0, max(r2_scores) * 1.2)
        for i, v in enumerate(r2_scores):
            axes[0].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

        # RMSE comparison
        rmse_scores = [self.results[m]['test_rmse'] for m in models]
        axes[1].bar(models, rmse_scores, color=['#3498db', '#2ecc71', '#e74c3c'])
        axes[1].set_ylabel('RMSE (%)', fontsize=12)
        axes[1].set_title('Model Performance Comparison (RMSE)', fontsize=14, fontweight='bold')
        for i, v in enumerate(rmse_scores):
            axes[1].text(i, v + 0.1, f'{v:.2f}', ha='center', fontweight='bold')

        plt.tight_layout()
        plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
        print("   ✓ Saved: model_comparison.png")
        plt.close()

        # Figure 2: Feature Importance (Random Forest)
        fig, ax = plt.subplots(figsize=(10, 8))

        rf_importance = pd.DataFrame({
            'Feature': self.feature_names,
            'Importance': self.models['Random Forest'].feature_importances_
        }).sort_values('Importance', ascending=False).head(15)

        # Clean feature names for display
        rf_importance['Feature_Clean'] = rf_importance['Feature'].str.replace('_', ' ').str.title()

        ax.barh(rf_importance['Feature_Clean'], rf_importance['Importance'], color='#2ecc71')
        ax.set_xlabel('Importance', fontsize=12, fontweight='bold')
        ax.set_title('Random Forest: Top 15 Feature Importance', fontsize=14, fontweight='bold')
        ax.invert_yaxis()

        plt.tight_layout()
        plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
        print("   ✓ Saved: feature_importance.png")
        plt.close()

        # Figure 3: Actual vs Predicted (Random Forest)
        fig, ax = plt.subplots(figsize=(8, 8))

        y_pred = self.results['Random Forest']['predictions']

        ax.scatter(self.y_test, y_pred, alpha=0.5, s=50, edgecolors='black', linewidth=0.5)

        # Perfect prediction line
        min_val = min(self.y_test.min(), y_pred.min())
        max_val = max(self.y_test.max(), y_pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')

        ax.set_xlabel('Actual Volatility Change (%)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Predicted Volatility Change (%)', fontsize=12, fontweight='bold')
        ax.set_title('Random Forest: Actual vs Predicted', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Add R² to plot
        r2 = self.results['Random Forest']['test_r2']
        ax.text(0.05, 0.95, f'R² = {r2:.3f}', transform=ax.transAxes,
                fontsize=14, fontweight='bold', verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.tight_layout()
        plt.savefig('actual_vs_predicted.png', dpi=300, bbox_inches='tight')
        print("   ✓ Saved: actual_vs_predicted.png")
        plt.close()

        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        residuals = self.y_test - y_pred

        axes[0].hist(residuals, bins=30, edgecolor='black', alpha=0.7, color='#3498db')
        axes[0].set_xlabel('Residuals (%)', fontsize=12)
        axes[0].set_ylabel('Frequency', fontsize=12)
        axes[0].set_title('Residual Distribution', fontsize=14, fontweight='bold')
        axes[0].axvline(0, color='red', linestyle='--', linewidth=2)

        # Residual plot
        axes[1].scatter(y_pred, residuals, alpha=0.5, s=50, edgecolors='black', linewidth=0.5)
        axes[1].axhline(0, color='red', linestyle='--', linewidth=2)
        axes[1].set_xlabel('Predicted Volatility Change (%)', fontsize=12)
        axes[1].set_ylabel('Residuals (%)', fontsize=12)
        axes[1].set_title('Residual Plot', fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('residual_analysis.png', dpi=300, bbox_inches='tight')
        print("   ✓ Saved: residual_analysis.png")
        plt.close()

    def save_results(self):
        """Save all results to files."""
        print("\n9. Saving results...")

        # Save predictions
        predictions_df = self.data[['state_fp', 'county_fp', 'county_name', 'state_name',
                                     'crop', 'yield_cv_change', 'predicted_cv_change',
                                     'predicted_high_risk']].copy()
        predictions_df.to_csv('model_predictions.csv', index=False)
        print("   ✓ Saved: model_predictions.csv")

        # Save model comparison
        comparison = pd.DataFrame(self.results).T
        comparison.to_csv('model_comparison_metrics.csv')
        print("   ✓ Saved: model_comparison_metrics.csv")

        # Save feature importance
        rf_importance = pd.DataFrame({
            'Feature': self.feature_names,
            'RF_Importance': self.models['Random Forest'].feature_importances_,
            'XGB_Importance': self.models['XGBoost'].feature_importances_
        }).sort_values('RF_Importance', ascending=False)
        rf_importance.to_csv('feature_importance.csv', index=False)
        print("   ✓ Saved: feature_importance.csv")

    def generate_report(self):
        """Generate comprehensive text report."""
        print("\n10. Generating summary report...")

        with open('modeling_report.txt', 'w') as f:
            f.write("="*80 + "\n")
            f.write("CROP YIELD VOLATILITY PREDICTION - MODELING REPORT\n")
            f.write("="*80 + "\n\n")

            # Dataset summary
            f.write("DATASET SUMMARY\n")
            f.write("-"*80 + "\n")
            f.write(f"Total samples: {len(self.X_train) + len(self.X_test)}\n")
            f.write(f"Training samples: {len(self.X_train)} (80%)\n")
            f.write(f"Test samples: {len(self.X_test)} (20%)\n")
            f.write(f"Number of features: {len(self.feature_names)}\n\n")

            # Model performance
            f.write("MODEL PERFORMANCE COMPARISON\n")
            f.write("-"*80 + "\n")
            for model_name, results in self.results.items():
                f.write(f"\n{model_name}:\n")
                f.write(f"  Test R²: {results['test_r2']:.4f}\n")
                f.write(f"  Test RMSE: {results['test_rmse']:.4f}%\n")
                f.write(f"  Test MAE: {results['test_mae']:.4f}%\n")
                f.write(f"  Cross-validation R²: {results['cv_r2_mean']:.4f} (+/- {results['cv_r2_std']:.4f})\n")

            # Feature importance
            f.write("\n\nFEATURE IMPORTANCE (Random Forest)\n")
            f.write("-"*80 + "\n")
            rf_imp = pd.DataFrame({
                'Feature': self.feature_names,
                'Importance': self.models['Random Forest'].feature_importances_
            }).sort_values('Importance', ascending=False)

            f.write(rf_imp.head(10).to_string(index=False))

            # Key insights
            f.write("\n\n\nKEY INSIGHTS\n")
            f.write("-"*80 + "\n")
            best_model = max(self.results.items(), key=lambda x: x[1]['test_r2'])[0]
            best_r2 = self.results[best_model]['test_r2']

            f.write(f"\n1. Best performing model: {best_model} (R² = {best_r2:.3f})\n")
            f.write(f"\n2. Top 3 predictive features:\n")
            for i, row in rf_imp.head(3).iterrows():
                f.write(f"   - {row['Feature']}: {row['Importance']:.3f}\n")

            f.write(f"\n3. Model can explain {best_r2*100:.1f}% of yield volatility variation\n")

            high_risk_count = (self.data['predicted_high_risk'] == True).sum()
            f.write(f"\n4. Predicted {high_risk_count} county-crop combinations as high-risk\n")

        print("   ✓ Saved: modeling_report.txt")


def main():
    """Main execution function."""

    # Initialize predictor
    predictor = VolatilityPredictor('/content/drive/MyDrive/project files/volatility_final_analysis.csv')

    # Prepare features
    X, y = predictor.prepare_features()

    # Train all models
    predictor.train_models()

    # Compare models
    comparison = predictor.compare_models()

    # Analyze feature importance
    rf_imp, xgb_imp = predictor.analyze_feature_importance()

    # Analyze linear coefficients
    coefficients = predictor.analyze_linear_coefficients()

    # Predict high-risk counties
    high_risk = predictor.predict_high_risk_counties(threshold=5.0)

    # Create visualizations
    predictor.create_visualizations()

    # Save results
    predictor.save_results()

    # Generate report
    predictor.generate_report()

    print("\n" + "="*80)
    print("MODELING COMPLETE!")
    print("="*80)
    print("\nGenerated files:")
    print("  1. model_comparison.png - Model performance comparison")
    print("  2. feature_importance.png - Top features driving volatility")
    print("  3. actual_vs_predicted.png - Prediction accuracy visualization")
    print("  4. residual_analysis.png - Model diagnostic plots")
    print("  5. model_predictions.csv - Predictions for all counties")
    print("  6. model_comparison_metrics.csv - Detailed metrics")
    print("  7. feature_importance.csv - Feature rankings")
    print("  8. modeling_report.txt - Comprehensive text report")
    print("="*80 + "\n")

    return predictor, comparison, rf_imp, high_risk


if __name__ == "__main__":
    predictor, comparison, rf_imp, high_risk = main()

CROP YIELD VOLATILITY PREDICTION MODEL

1. Loading data...
   Loaded 3406 records

2. Preparing features...
   Removed 218 rows with missing values
   Final dataset: 3188 samples
   Features: 13
   Target: yield_cv_change (volatility change %)
   Training samples: 2550
   Test samples: 638

3. Training models...
--------------------------------------------------------------------------------

   Model 1: Linear Regression (Baseline)
      Train R²: 0.6042
      Test R²:  0.5092
      Test RMSE: 6.3219
      Test MAE:  4.2791
      CV R² (5-fold): 0.5930 (+/- 0.0313)

   Model 2: Random Forest (Primary Model)
      Train R²: 0.8491
      Test R²:  0.5473
      Test RMSE: 6.0715
      Test MAE:  4.0185
      CV R² (5-fold): 0.6161 (+/- 0.0223)

   Model 3: XGBoost (Advanced Model)
      Train R²: 0.8820
      Test R²:  0.5657
      Test RMSE: 5.9465
      Test MAE:  3.9319
      CV R² (5-fold): 0.6364 (+/- 0.0174)

-------------------------------------------------------------------------

In [ ]:
import pickle

# Export XGBoost model
with open('xgboost_model.pkl', 'wb') as f:
    pickle.dump(predictor.models['XGBoost'], f)

# Download it
from google.colab import files
files.download('xgboost_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle

with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(predictor.models['Random Forest'], f)

from google.colab import files
files.download('random_forest_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/drive/MyDrive/project files/volatility_final_analysis.csv